# Zczytanie z wikisource:
https://pl.wikisource.org/wiki/Kategoria:Polscy_poeci

In [1]:
import pandas as pd
import requests
import os
import io
import random
import time
import re

from bs4 import BeautifulSoup

In [2]:
#zczytanie wszystkich linków w zbiorach wierszy
page_list =["https://pl.wikisource.org/wiki/Kategoria:Polscy_poeci",
           "https://pl.wikisource.org/w/index.php?title=Kategoria:Polscy_poeci&pagefrom=Laskowski%2C+Kazimierz%0AKazimierz+Laskowski#mw-pages",
           "https://pl.wikisource.org/w/index.php?title=Kategoria:Polscy_poeci&pagefrom=Wolski%2C+W%C5%82odzimierz%0AW%C5%82odzimierz+Wolski#mw-pages"]

url_autors = []

start = time.time()

for page_url in page_list:
    #wybranie strony do szukania linków z autorami
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for a in soup.find_all('a', href=True):
        if a['href'][0:11]=='/wiki/Autor':
            url_autors.append(a['href'])
            
print(f'Pobrałem {len(url_autors)} autorów w ciągu {round(time.time()-start,1)} sekund')

Pobrałem 427 autorów w ciągu 1.4 sekund


## Odpalenie na całości

In [3]:
i = 1 # to iteracji kolejnych autorów

for url_autor in url_autors:
    start = time.time()
    autor = url_autor.replace('/wiki/Autor:','')

    # stworzenie folderu autora jesli nie istnieje
    directory = f'./data_raw/{autor}/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    url = 'https://pl.m.wikisource.org'+url_autor.replace('Autor','Kategoria') #ta zmiana daje prostszą stronę

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        #znalezienie wierszy ze strony autora
        poems_url = []
        for a in soup.find_all('a', href=True):
            if '/wiki/Autor:' not in a['href'] and '/wiki/Kategoria:' not in a['href']:
                poems_url.append(a['href'])

        #przycięcie do samych tekstów i wywalenie zbędnych linków
        list_index_from = [idx for idx, s in enumerate(poems_url) if '&from=%C5%BB' in s][0]
        poems_url = poems_url[list_index_from+1:]
        
        poems_url = [poems for poems in poems_url if '/w/index.php?title=Kategoria:' not in poems] 
        
        list_index_to = [idx for idx, s in enumerate(poems_url) if '/wiki/' not in s][0]
        poems_url = poems_url[:list_index_to]
     
    except:
        pass  

    # pętla do pobraniawszystkich znalezionych wierszy z url
    for poem in poems_url:
        title = poem.replace('/wiki/','')

        try:
            page = requests.get('https://pl.m.wikisource.org'+poem)
            soup = BeautifulSoup(page.content, 'html.parser')

            # zamiana break lines na nową linie
            for br in soup.find_all("br"):
                br.replace_with("\n")

            #znajdowanie tekstu
            text = soup.find_all('p')

            #czyszczenie
            clean_text = re.sub('<.*?>', '', str(text)) #usunięcie rzeczy htmlowych pomiędzy znakami < oraz >

            #zapisanie pliku
            with io.open(directory+title.replace('/',' ')+'.txt', "w", encoding="utf-8") as f:
                f.write(str(clean_text))
                
        except:
            pass    
    
    print(f'Autor {i} z {len(url_autors)}: {autor}; pobrano {len(poems_url)} dzieł w ciągu {round(time.time()-start,1)} sekund')
    print(url)
    i = i+1

Autor 1 z 427: Piotr_Adamczyk; pobrano 2 dzieł w ciągu 0.8 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Piotr_Adamczyk
Autor 2 z 427: Bogus%C5%82aw_Adamowicz; pobrano 15 dzieł w ciągu 3.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Bogus%C5%82aw_Adamowicz
Autor 3 z 427: Jan_Chrzciciel_Albertrandi; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Jan_Chrzciciel_Albertrandi
Autor 4 z 427: Alx_z_Poewiki; pobrano 4 dzieł w ciągu 1.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Alx_z_Poewiki
Autor 5 z 427: Jan_Ancuta; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Jan_Ancuta
Autor 6 z 427: W%C5%82adys%C5%82aw_Ludwik_Anczyc; pobrano 21 dzieł w ciągu 4.3 sekund
https://pl.m.wikisource.org/wiki/Kategoria:W%C5%82adys%C5%82aw_Ludwik_Anczyc
Autor 7 z 427: Karol_Antoniewicz; pobrano 158 dzieł w ciągu 28.6 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Karol_Antoniewicz
Autor 8 z 427: Franciszka_Arnsztajnowa; pobrano 18 

Autor 61 z 427: Aleksander_Chodecki; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Aleksander_Chodecki
Autor 62 z 427: Aleksander_Chod%C5%BAko; pobrano 11 dzieł w ciągu 2.3 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Aleksander_Chod%C5%BAko
Autor 63 z 427: Edmund_Chojecki; pobrano 2 dzieł w ciągu 0.8 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Edmund_Chojecki
Autor 64 z 427: Stanis%C5%82aw_Ciesielczuk; pobrano 91 dzieł w ciągu 16.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Stanis%C5%82aw_Ciesielczuk
Autor 65 z 427: Adam_Czahrowski; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Adam_Czahrowski
Autor 66 z 427: Antoni_Czajkowski; pobrano 5 dzieł w ciągu 1.3 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Antoni_Czajkowski
Autor 67 z 427: Celestyn_Czaplic; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Celestyn_Czaplic
Autor 68 z 427: Maria_Czarnowska-Loevy; pobrano 3 dzieł 

Autor 122 z 427: Konstanty_G%C3%B3rski; pobrano 5 dzieł w ciągu 1.4 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Konstanty_G%C3%B3rski
Autor 123 z 427: Sebastian_Grabowiecki; pobrano 6 dzieł w ciągu 1.5 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Sebastian_Grabowiecki
Autor 124 z 427: Antoni_Grabowski; pobrano 6 dzieł w ciągu 1.4 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Antoni_Grabowski
Autor 125 z 427: Bronis%C5%82aw_Teodor_Grabowski; pobrano 14 dzieł w ciągu 2.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Bronis%C5%82aw_Teodor_Grabowski
Autor 126 z 427: Stanis%C5%82aw_Grochowiak; pobrano 200 dzieł w ciągu 39.6 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Stanis%C5%82aw_Grochowiak
Autor 127 z 427: Stanis%C5%82aw_Grochowski; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Stanis%C5%82aw_Grochowski
Autor 128 z 427: Maria_Grossek-Korycka; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Maria_Grosse

Autor 181 z 427: J%C3%B3zef_Ko%C5%9Bcielski; pobrano 9 dzieł w ciągu 2.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:J%C3%B3zef_Ko%C5%9Bcielski
Autor 182 z 427: W%C5%82adys%C5%82aw_August_Ko%C5%9Bcielski; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:W%C5%82adys%C5%82aw_August_Ko%C5%9Bcielski
Autor 183 z 427: Franciszek_Kowalski; pobrano 38 dzieł w ciągu 7.5 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Franciszek_Kowalski
Autor 184 z 427: Kajetan_Ko%C5%BAmian; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Kajetan_Ko%C5%BAmian
Autor 185 z 427: Ignacy_Krasicki; pobrano 92 dzieł w ciągu 17.1 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Ignacy_Krasicki
Autor 186 z 427: Zygmunt_Krasi%C5%84ski; pobrano 49 dzieł w ciągu 9.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Zygmunt_Krasi%C5%84ski
Autor 187 z 427: J%C3%B3zef_Ignacy_Kraszewski; pobrano 200 dzieł w ciągu 44.4 sekund
https://pl.m.wikisource.org/wiki/Ka

Autor 240 z 427: Franciszek_Dzier%C5%BCykraj_Morawski; pobrano 15 dzieł w ciągu 3.6 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Franciszek_Dzier%C5%BCykraj_Morawski
Autor 241 z 427: Hieronim_Morsztyn; pobrano 7 dzieł w ciągu 1.6 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Hieronim_Morsztyn
Autor 242 z 427: Jan_Andrzej_Morsztyn; pobrano 200 dzieł w ciągu 40.0 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Jan_Andrzej_Morsztyn
Autor 243 z 427: Micha%C5%82_Morzkowski; pobrano 1 dzieł w ciągu 0.5 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Micha%C5%82_Morzkowski
Autor 244 z 427: Irena_Mrozowicka; pobrano 2 dzieł w ciągu 0.8 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Irena_Mrozowicka
Autor 245 z 427: Daniel_Naborowski; pobrano 31 dzieł w ciągu 6.0 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Daniel_Naborowski
Autor 246 z 427: Ignacy_Nagurczewski; pobrano 3 dzieł w ciągu 0.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Ignacy_Nagurczewski
Autor 247 z 427

Autor 300 z 427: Jacek_Przybylski; pobrano 4 dzieł w ciągu 1.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Jacek_Przybylski
Autor 301 z 427: Stanis%C5%82aw_Przybyszewski; pobrano 35 dzieł w ciągu 8.3 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Stanis%C5%82aw_Przybyszewski
Autor 302 z 427: Melchior_Pud%C5%82owski; pobrano 185 dzieł w ciągu 33.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Melchior_Pud%C5%82owski
Autor 303 z 427: Teresa_Radziewicz; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Teresa_Radziewicz
Autor 304 z 427: Ludwik_Edward_Rajszel; pobrano 9 dzieł w ciągu 2.0 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Ludwik_Edward_Rajszel
Autor 305 z 427: Miko%C5%82aj_Rej; pobrano 199 dzieł w ciągu 36.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Miko%C5%82aj_Rej
Autor 306 z 427: Zofia_Reutt-Witkowska; pobrano 3 dzieł w ciągu 1.0 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Zofia_Reutt-Witkowska
Autor 307 z 427: Tymote

Autor 358 z 427: Maciej_Szukiewicz; pobrano 8 dzieł w ciągu 1.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Maciej_Szukiewicz
Autor 359 z 427: Bogus%C5%82aw_Szul-Skj%C3%B6ldkrona; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Bogus%C5%82aw_Szul-Skj%C3%B6ldkrona
Autor 360 z 427: Teofil_Szumski; pobrano 2 dzieł w ciągu 0.9 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Teofil_Szumski
Autor 361 z 427: Edward_Szyma%C5%84ski; pobrano 16 dzieł w ciągu 3.1 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Edward_Szyma%C5%84ski
Autor 362 z 427: Szymon_Szymonowic; pobrano 74 dzieł w ciągu 14.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Szymon_Szymonowic
Autor 363 z 427: Julian_%C5%9Awi%C4%99cicki; pobrano 42 dzieł w ciągu 8.2 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Julian_%C5%9Awi%C4%99cicki
Autor 364 z 427: Wac%C5%82aw_%C5%9Awi%C4%99cicki; pobrano 2 dzieł w ciągu 0.8 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Wac%C5%82aw_%C5%9Awi

Autor 416 z 427: Andrzej_Zbylitowski; pobrano 9 dzieł w ciągu 2.1 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Andrzej_Zbylitowski
Autor 417 z 427: Emil_Zegad%C5%82owicz; pobrano 35 dzieł w ciągu 6.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Emil_Zegad%C5%82owicz
Autor 418 z 427: Adam_Zeme%C5%82ka; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Adam_Zeme%C5%82ka
Autor 419 z 427: Wanda_Ziele%C5%84czyk; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Wanda_Ziele%C5%84czyk
Autor 420 z 427: Gustaw_Zieli%C5%84ski; pobrano 5 dzieł w ciągu 1.3 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Gustaw_Zieli%C5%84ski
Autor 421 z 427: J%C3%B3zef_Bart%C5%82omiej_Zimorowic; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:J%C3%B3zef_Bart%C5%82omiej_Zimorowic
Autor 422 z 427: Szymon_Zimorowic; pobrano 2 dzieł w ciągu 0.7 sekund
https://pl.m.wikisource.org/wiki/Kategoria:Szymon_Zimorowic
Autor 423 z

Zauważyłem, że jak ktoś ma 200 dzieł, to okazazuje się, że często ma więcej :) </br>

Zebrałem wobec tego ręcznie, bo ciężko samemu wymyślić link :(

In [4]:
url_autors_additional = [
  'https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Be%C5%82za&pagefrom=Modlitwa+polskiego+dziecka#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:J%C3%B3zef_Birkenmajer&pagefrom=Sprawa+opiecz%C4%99towana#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Leopold_Blumental&pagefrom=Porz%C4%85dek#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Julian_Ejsmond&pagefrom=Wr%C3%B3ble+%C5%9Bwiegoty#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Aleksander_Fredro&pagefrom=W+dzie%C5%84+imienin+wnuka#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Wiktor_Gomulicki&pagefrom=Tajemnica+%28Gomulicki%2C+1866%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Stanis%C5%82aw_Grochowiak&pagefrom=W+g%C5%82%C4%85b+kalendarza+niech+wiod%C4%85+imiona...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Stanis%C5%82aw_Jachowicz&pagefrom=Ucze%C5%84+niepilny#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Jan_Kasprowicz&pagefrom=Rozkosz+mi%C5%82o%C5%9Bci+%28Rossetti%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Ludwik_Kondratowicz&pagefrom=Oda+VII.+Wiekuista+pami%C4%99%C4%87+cieniom+ojca+i+syna#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Konopnicka&pagefrom=Hejna%C5%82+%28Bij+nam...%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Konopnicka&pagefrom=Na+normandzkim+brzegu%2FCi+mali#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Konopnicka&pagefrom=Pod+Iganiami#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Konopnicka&pagefrom=Tego+si%C4%99+strze%C5%BC...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:J%C3%B3zef_Ignacy_Kraszewski&pagefrom=Nam...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:J%C3%B3zef_Ignacy_Kraszewski&pagefrom=Wiosna+zwodnica...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Aleksander_Kraushar&pagefrom=Wiatr+szumi%C4%85c#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Antoni_Kucharczyk&pagefrom=P%C4%99dzi%C4%87+bied%C4%99%21#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Antoni_Lange&pagefrom=Pie%C5%9B%C5%84+awanturnik%C3%B3w+morskich+z+XVI+wieku#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Antoni_Lange&pagefrom=W%C4%99gierska+pie%C5%9B%C5%84+junacza#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Boles%C5%82aw_Le%C5%9Bmian&pagefrom=Urszula+Kochanowska#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Jan_Andrzej_Morsztyn&pagefrom=Na+to%C5%BC+%28Udatna+n%C3%B3%C5%BCko%21...%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Cyprian_Kamil_Norwid&pagefrom=Nad+grobem+Julji+Capuleti+we+Weronie#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Artur_Oppman&pagefrom=Oberek+%281903%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Orkan&pagefrom=O+Tetmajerowem+%E2%80%9ESkalnem+Podhalu%E2%80%9C#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Orkan&pagefrom=Z+dalekich+l%C4%85d%C3%B3w%2C+z+dalekich+dr%C3%B3g...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Pawlikowska-Jasnorzewska&pagefrom=Kwiatki+z+Waterloo+%28Pawlikowska-Jasnorzewska%2C+1926%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Maria_Pawlikowska-Jasnorzewska&pagefrom=R%C3%B3%C5%BCe+dla+Safony%2FIII#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Wac%C5%82aw_Potocki&pagefrom=Rozwi%C4%85zana+miot%C5%82a#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Zygmunt_R%C3%B3%C5%BCycki&pagefrom=Wied%C5%BAma+%28R%C3%B3%C5%BCycki%2C+1911%29#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Juliusz_S%C5%82owacki&pagefrom=%C5%9Amier%C4%87%2C+co+trzyna%C5%9Bcie+lat+sta%C5%82a...#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Tarnowski&pagefrom=Krzy%C5%BC+przy+Drodze#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Tarnowski&pagefrom=Praxytel+i+Fryne#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Kazimierz_Przerwa-Tetmajer&pagefrom=Preludja#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Jerzy_%C5%BBu%C5%82awski&pagefrom=O+idei+mickiewiczowskiej#mw-pages'
, 'https://pl.m.wikisource.org/w/index.php?title=Kategoria:Jerzy_%C5%BBu%C5%82awski&pagefrom=Zwyci%C4%99zca+%28Nie+m%C3%B3w+o+sobie...%29#mw-pages'
]

In [6]:
i = 1 # to iteracji kolejnych autorów

for url_autor in url_autors_additional:
    start = time.time()
    autor = url_autor.replace('https://pl.m.wikisource.org/w/index.php?title=Kategoria:','')
    autor = autor[:autor.find('&pagefrom')]

    # stworzenie folderu autora jesli nie istnieje
    directory = f'./data_raw2/{autor}/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    page = requests.get(url_autor)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        #znalezienie wierszy ze strony autora
        poems_url = []
        for a in soup.find_all('a', href=True):
            if '/wiki/Autor:' not in a['href'] and '/wiki/Kategoria:' not in a['href']:
                poems_url.append(a['href'])

        #przycięcie do samych tekstów i wywalenie zbędnych linków
        list_index_from = [idx for idx, s in enumerate(poems_url) if '&from=%C5%BB' in s][0]
        poems_url = poems_url[list_index_from+1:]
        
        poems_url = [poems for poems in poems_url if '/w/index.php?title=Kategoria:' not in poems] 
        
        list_index_to = [idx for idx, s in enumerate(poems_url) if '/wiki/' not in s][0]
        poems_url = poems_url[:list_index_to]
     
    except:
        pass  

    # pętla do pobraniawszystkich znalezionych wierszy z url
    for poem in poems_url:
        title = poem.replace('/wiki/','')

        try:
            page = requests.get('https://pl.m.wikisource.org'+poem)
            soup = BeautifulSoup(page.content, 'html.parser')

            # zamiana break lines na nową linie
            for br in soup.find_all("br"):
                br.replace_with("\n")

            #znajdowanie tekstu
            text = soup.find_all('p')

            #czyszczenie
            clean_text = re.sub('<.*?>', '', str(text)) #usunięcie rzeczy htmlowych pomiędzy znakami < oraz >

            #zapisanie pliku
            with io.open(directory+title.replace('/',' ')+'.txt', "w", encoding="utf-8") as f:
                f.write(str(clean_text))
                
        except:
            pass    
    
    print(f'Autor {i} z {len(url_autors_additional)}: {autor}; pobrano {len(poems_url)} dzieł w ciągu {round(time.time()-start,1)} sekund')
    print(url_autor)
    i = i+1

Autor 1 z 36: W%C5%82adys%C5%82aw_Be%C5%82za; pobrano 149 dzieł w ciągu 27.5 sekund
https://pl.m.wikisource.org/w/index.php?title=Kategoria:W%C5%82adys%C5%82aw_Be%C5%82za&pagefrom=Modlitwa+polskiego+dziecka#mw-pages
Autor 2 z 36: J%C3%B3zef_Birkenmajer; pobrano 89 dzieł w ciągu 17.1 sekund
https://pl.m.wikisource.org/w/index.php?title=Kategoria:J%C3%B3zef_Birkenmajer&pagefrom=Sprawa+opiecz%C4%99towana#mw-pages
Autor 3 z 36: Leopold_Blumental; pobrano 107 dzieł w ciągu 20.7 sekund
https://pl.m.wikisource.org/w/index.php?title=Kategoria:Leopold_Blumental&pagefrom=Porz%C4%85dek#mw-pages
Autor 4 z 36: Julian_Ejsmond; pobrano 13 dzieł w ciągu 3.0 sekund
https://pl.m.wikisource.org/w/index.php?title=Kategoria:Julian_Ejsmond&pagefrom=Wr%C3%B3ble+%C5%9Bwiegoty#mw-pages
Autor 5 z 36: Aleksander_Fredro; pobrano 36 dzieł w ciągu 7.4 sekund
https://pl.m.wikisource.org/w/index.php?title=Kategoria:Aleksander_Fredro&pagefrom=W+dzie%C5%84+imienin+wnuka#mw-pages
Autor 6 z 36: Wiktor_Gomulicki; pobrano 